In [1]:
from pinecone import Pinecone, ServerlessSpec
import os
import pandas as pd
from time import time
import dotenv
import uuid
dotenv.load_dotenv()

/opt/anaconda3/envs/ai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
embedding_model= os.getenv("EMBEDDING_MODEL")
pinecone_api_key = os.getenv("PINECONE_API_KEY")

In [3]:
print(embedding_model)

sentence-transformers/all-MiniLM-L6-v2


In [4]:
pc = Pinecone(api_key=pinecone_api_key)

In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

embedding_model = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=os.getenv("GEMINI_API_KEY")
)

In [6]:
len(embedding_model.embed_query("hlo"))


768

In [7]:
df = pd.read_json("/Users/pandhari/Desktop/Coffee_Shop_ChatBot/Python_Backend/Backend/Data/products_data/products.jsonl" , lines=True)

In [8]:
df.head(2)

,name,category,description,ingredients,price,rating,image_path
0,Cappuccino,Coffee,A rich and creamy cappuccino made with freshly...,"[Espresso, Steamed Milk, Milk Foam]",4.50,4.7,cappuccino.jpg
1,Jumbo Savory Scone,Bakery,"Deliciously flaky and buttery, this jumbo savo...","[Flour, Butter, Cheese, Herbs, Baking Powder, ...",3.25,4.3,SavoryScone.webp


In [9]:
df['text'] = (
    df['name'] + ":\n" +
    df['description'] + "\n" +
    "Ingredients: " + df['ingredients'].astype(str) + "\n" +
    "Price: $" + df['price'].astype(str) + "\n" +
    "Rating: " + df['rating'].astype(str)
)

In [10]:
df['text'].head()

0    Cappuccino:\nA rich and creamy cappuccino made...
1    Jumbo Savory Scone:\nDeliciously flaky and but...
2    Latte:\nSmooth and creamy, our latte combines ...
3    Chocolate Chip Biscotti:\nCrunchy and delightf...
4    Espresso shot:\nA bold shot of rich espresso, ...
Name: text, dtype: object

In [11]:
# Initialize Pinecone
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index_name = "coffee-products"
if index_name not in pc.list_indexes().names():
    pc.create_index(
        index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
index = pc.Index(index_name)

In [12]:
for _, row in df.iterrows():
    text_to_embed = row["text"]
    embedding = embedding_model.embed_query(text_to_embed)

    metadata = {
        "text": text_to_embed,
        "name": row["name"],
        "category": row["category"],
        "price": float(row["price"]),
        "rating": float(row["rating"])
    }

    index.upsert([
        {
            "id": str(uuid.uuid4()),
            "values": embedding,
            "metadata": metadata
        }
    ])